# SIXT33N Project
## Phase 5: Advanced Controls

### EECS16B: Designing Information Devices and Systems II, Spring 2021

Written by Vivek Athalye and Andrew Blatner (2017). Revised by Peter Schafhalter (2019).

## Table of Contents

* [Introduction](#intro)
* [Lab Note](https://drive.google.com/file/d/1WPWbG77a4GgpUpSI-EbCotnJu1zbetZ3/view?usp=sharing)
* [Part 1: Launchpad Implementation of PCA Classify](#part1)
* [Part 2: Analyzing a Turn](#part2)
* [Part 3: Implementing Turns](#part3)
* [Part 4: Correcting Mechanical Errors](#part4)

<a id='intro'></a>
## Introduction 

In the previous phase, you recorded data and found the PCA vectors that you used to classify a testing subset of your prerecorded data. In this phase, you will implement a live classification of the commands you say into the microphone using the Launchpad.

We will also be implementing turning in this phase. The car needs to follow 4 different commands: turn left, turn right, drive straight far, and drive straight close. In the Controls phase, we implemented a closed loop controller that succeeded in controlling the car to drive in a straight line.

Recall that our control scheme for SIXT33N modeled each wheel in the following way: $d_R[n+1] = d_R[n] + \theta_R u_R[n] - \beta_R$.

The open loop model has an input $u[n]$, corresponding to the PWM level, and outputs the distance for that wheel. We implemented a closed-loop controller by using the desired velocity and the difference between the wheel distances, $\delta[n] = d_L[n] - d_R[n]$.

We will modify that closed loop control scheme to enable the car to turn.

<a id='part4'></a>
## <span style="color:navy">Part 1: Launchpad Implementation of PCA Classify</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB


This section will walk you through implementing your classification algorithm on the Launchpad. In addition, you will need to transfer the PCA vectors and projected mean to the Launchpad. **You will be copying and pasting the code from the Appendix of the PCA notebook into `classify.ino.`**

Your first task will be to implement your <b>data processing</b> and <b>classification</b> (just the projection, not the PCA) in the Launchpad sketch <b>`classify.ino`</b>. Since Energia does not have as many in-built functions as Python, you will have to write out the functions yourself. For example, a dot product should be written as:

```C
float dot_product_result = 0;
for (i = 0; i < LENGTH; i++) {
    dot_product_result += vector1[i] * vector2[i];
}
```
where `dot_product_result` is the result of the dot product, and `vector1` and `vector2` are the two vectors you're taking the dot product of.

NOTE: The coding language is a derivative of C/C++, so you need to follow C syntax! i.e. declaring variables before using them, using `{` and `}` to denote the start and end of a for loop, adding a `;` to the ends of lines, etc. You can reference the existing code for examples of how to use the proper syntax.

For debugging purposes, you can add print statements to the code. Printing to Energia's serial monitor looks like the line below.

`Serial.println("I'm being printed!");`

There are 3 code blocks (`PCA1/2/3`) that you need to modify. <b>You should not have to change anything else outside these marked code blocks, unless you're using a different pin than the default `P6.0` pin.</b> 

**`CODE BLOCK PCA1`**
- Copy the `SNIPPET_SIZE`, `PRELENGTH` and `THRESHOLD` from the appendix section of the PCA ipynb.
- Read the following to set `EUCLIDEAN_THRESHOLD` and `LOUDNESS_THRESHOLD`.

**`EUCLIDEAN_THRESHOLD` filters the classification depending on a sample's distance to the closest centroid.** If the L2 norm (distance) is larger than the threshold, your classification algorithm should simply ignore it and wait for the next sample. Look at the plot of your data clusters and the centroids from the PCA Notebook and approximate a radius around the centroids that capture most of the data. **Try to be conservative - it's better to not classify than to misclassify.**

**`LOUDNESS_THRESHOLD` filters the classification depending on the amplitude of the recorded data.** If the recorded data is too soft, we do not want to classify it as it is probably noise. Since the loudness unit is arbitrary, start by using `700`. Later, if the Launchpad classifies noise, increase this constant. If it misses a lot of speech (i.e. thinks your word is noise), decrease this constant. This variable is used internally in the enveloping function.

**`CODE BLOCK PCA2`**
- Copy the PCA vectors, projected mean vector, and centroids from the appendix section of the PCA ipynb.
- If you are using 3 principal components, add a new `pca_vec3` array. 
- Using more principal components increases the dimensionality of the centroids and projections, but also consumes more of the limited memory on the Launchpad.

**`CODE BLOCK PCA3`**
- This is the actual classification algorithm.
- Before this block, the call to `envelope` leaves the data vector of your recording in the array called `result`.
- Project this data onto your new PCA basis.
    - Use just one loop to project your recorded data vector onto both (or all 3, if you're using 3) of your PCA vectors.
    - Use the variables `proj1` and `proj2` to store the projection results.
    - If you're using 3 vectors, create a variable `proj3` in the same way `proj1` and `proj2` are defined.
- Demean the projection.
    - Remember that we *demean after projecting* to save memory on the Launchpad. Instead of finding $x - \bar{x}$ and then projecting onto the PCA basis, we first project and then demean, using $ y = x_{\text{proj}} - \bar{x}_{\text{proj}} $, where $x_{\text{proj}}$ is the projection of your data vector onto the PCA basis, and $\bar{x}_{\text{proj}}$ is the projection of your mean vector onto the PCA basis.
- Classify the projections using the centroids.
    - Find the distance between the projected data point and each centroid using the function `l2_norm` (for 2 principal components) or `l2_norm3` (for 3 principal components). Look up the function definition in the sketch.
    - Out of the 4 centroids, find the one with the smallest L2 norm.
    - Verify this distance is less than `EUCLIDEAN_THRESHOLD`. If it's not, print out an error statement saying this threshold was not satisfied so you have an easier time debugging.
- Print the classification to the serial monitor. The baud rate for this program is 38400, so make sure you change the baud rate of your serial monitor accordingly!

Before testing the code, probe the mic board's output with your multimeter and make sure that it still is centered around 1.65V. Now upload the sketch, (re)open the serial monitor, and press the reset button. Say your word and the Launchpad should recognize it! Try to get a reasonable accuracy (like at least 80-90%); it's okay if it misclassifies occasionally, but it should be accurate in general!

**<span style="color:red">If the Launchpad does not classify as well as you think it should, remember to play with the `EUCLIDEAN_THRESHOLD` and `LOUDNESS_THRESHOLD` variables.</span>** To debug the sketch, you can also print out any of the variables you have used, like the distance to the closest centroid.

Voila! Your SIXT33N can recognize your words!

**<span style="color:red">Summary TO DO</span>** 
- **<span style="color:red">Fill in `CODE BLOCK PCA1`: Fill out `SNIPPET_SIZE`, `PRELENGTH`, `THRESHOLD`, `EUCLIDEAN_THRESHOLD` and `LOUDNESS_THRESHOLD`</span>** 
- **<span style="color:red">Fill in `CODE BLOCK PCA2`: Copy the principal components, mean vector and centroids from the IPython notebook</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA3`: Do the actual classification.</span>**

<a id='part2'></a>
## <span style="color:navy">Part 2: Analyzing a Turn</span>

### Materials
- SIXT33N Car
- Closed-loop control scheme

First, verify that your closed loop control scheme is still functional. Load `closed_loop.ino` onto your launchpad and make sure your car drives straight.

### We want to turn with a specific radius.  What open-loop control input would lead to turning?  
During open-loop control, we sought to drive the car straight at our desired velocity $v^*$.  

We used our model of the car ($\theta$, $\beta$) to calculate the open-loop control input which drives the car straight in the absence of perturbations:  

$$u^{OL}_L = \frac{v^*+\beta_L}{\theta_L} $$
$$u^{OL}_R = \frac{v^*+\beta_R}{\theta_R} $$

Is it possible to calculate an open-loop control input which would result in turning with a desired radius?


### We have been turning during Closed-loop Control
We can get a clue about turning by considering how we did closed-loop control. We perform feedback control of SIXT33N in order to minimize the difference between the two wheels ($\delta[n] = d_L[n] - d_R[n]$). When perturbations cause one wheel to get ahead of the other resulting in non-zero $\delta$, feedback control turns the car to correct the error.  

Let's walk through this.  

Our feedback control policy is: 
$$u_L[n] = u_L^{OL} - \frac{k_L}{\theta_L}\delta[n]$$

$$u_R[n] = u_R^{OL} + \frac{k_R}{\theta_R}\delta[n]$$


Let's say the right wheel has moved further than the left ($d_R[n] > d_L[n]$), resulting in a negative $\delta[n]$. A negative $\delta[n]$ results in a reduction of $u_R[n]$, and an increase of $u_L[n]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So $\delta \ne 0$ can be used to turn the car. Let's get more precise.  


### Turning via reference tracking

We would like the car to turn with a specified radius $r$ and linear velocity $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of wheel circumference.

To turn, we want $\delta$ to change at a particular rate. Without loss of generality, we'll analyze a right turn, corresponding to increasing $\delta$. For a left turn, we simply negate $\delta$. Our goal is to generate a reference from the desired $r$ and $v^*$ for the controller to follow. This reference will be a function of the controller's time-step.

We define the following variables:
- $n$ [time] - time-step
- $r$ [cm] - turn radius of the center of the car; 1 cm ~ 1 encoder tick
- $d$ [tick] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm ~ 1 encoder tick
- $\omega$ [rad/time] - angular velocity
- $\theta$ [rad] - angle traveled

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>



From this geometry, can you write $\delta[n]$ in the following form?
$$\delta[n] = f(r,v^*,l,n)$$

<h3><span style='color:red'>**Check your answer with the lab staff!**</span></h3>

Answer:

<a id='part3'></a>
## <span style="color:navy">Part 3: Implementing Turns</span>

### Materials
- SIXT33N Car
- Closed-loop control scheme
- Plan from Task 1

### Sampling periods 
 
In our implementation, the control loop and the data collection have different periods, $T_c$ and $T_d$ respectively, and therefore different sampling frequencies, $F_c$ and $F_d$, respectively.  $F_c$ is an integer multiple of $F_d$: $m=\frac{F_c}{F_d} \in \mathbb{Z}^{+}$

To ensure the car is travelling at the same ground velocity in both cases, we want to have a constant distance, measured in ticks, travelled per second, or $\frac{ticks}{s}$.

$F_c$, with units [$\frac{sample_c}{s}$], is the number of times the closed loop is sampled every second, and $F_d$, with units with units [$\frac{sample_d}{s}$], is the number of times data collection was sampled per second. Let $v^*_c$ be the velocity of closed loop, and $v^*$ be the velocity of the data collection. $v^*_c$ has the units of [$\frac{ticks}{sample_c}$] and $v^*$ has units [$\frac{ticks}{sample_d}$].

To get a constant $\frac{ticks}{s}$, we need $v^*_c * F_c = v^* * F_d$.

Solving for $v^*_c$, we get $v^*_c = v^* * \frac{F_d}{F_c} = \frac{v^*}{m}$

We use $T_c=100$ms, $T_d=500ms$, so $F_c=10$, $F_d=2$, so $m=5$.

**<span style='color:red'> In your driveStraight functions, use $\frac{v^*}{m}$ instead of $v^*$. Do not replace the original $v^*$ parameter defined in `CODE BLOCK CON1`!</span>**

### Implementation
- Open `turning.ino`.
- Copy the required code from `closed_loop.ino` into **`CODE BLOCK CON1/2/3`**
    - Make sure to update pin numbers if you've made changes.
    - Note that your `driveStraight` only has one parameter in this program unlike closed_loop.ino, so only copy over the body of the function!
    - Use $\frac{v^*}{m}$ instead of $v^*$ in your driveStraight functions!
- **Complete the function `delta_reference` in `CODE BLOCK CON4`**.
    - Look at `CODE BLOCK CON0` to see how this is used!
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR_WIDTH`, and `n` (the time step).
    - `delta_ss` is used in `CODE BLOCK CON0`, so don't include it here.
- Program a command string for your car to follow in **`PREPROGRAMMED PATH`**
    - Set `NUM_COMMANDS` to the number of commands you want your car to follow.
    - In the `run_times` and `drive_modes` arrays, add at least 8 commands you want your car to follow as well as the length associated with each command.
        - The number of elements in both `drive_modes` and `run_times` should be equal to `NUM_COMMANDS`
        - The values in `run_times` are in ms, so you should have run times that are at least 1000.
        - `drive_modes` can take on values: `[DRIVE_STRAIGHT, DRIVE_LEFT, DRIVE_RIGHT]`. Make sure to include at least one of each command in your path!

When you're done, try it out! SIXT33N should follow your preprogrammed commands with a stop between each stage. It's okay if SIXT33N does not turn exactly 90 degrees. If you want to adjust the turn you can change the turn radius or the turn's runtime until it only turns 90 degrees. Just make sure that it is clear when SIXT33N is turning, and when it is driving straight.

<a id='part4'></a>
## <span style="color:navy">Part 4: Correcting Mechanical Errors</span>

At this stage, your car may already go perfectly straight and turn left and right equally. However, some cars may not go quite straight even when both encoder measurements are exactly the same. This can be caused by mechanical issues such as axle wobble or mismatch in the wheel sizes. In this case, the car thinks it's driving straight, so it's the best we can do using just the sensors in our control system.

If the car is consistently turning slightly instead of driving straight, we can correct it by instructing the car to slightly turn in the opposite direction with a radius of `STRAIGHT_RADIUS`. 

**If you think your car could benefit from this, implement the function `straight_correction` in `CODE BLOCK CON5` using the variable `STRAIGHT_RADIUS`.**